
# FIne-tune on 3 epoch's

In [ ]:
!mkdir -p "mbart"
%cd mbart

/content/mbart


In [1]:
!pip install transformers
!pip install datasets
!pip install sacrebleu
!pip install sentencepiece
!pip install evaluate
!pip install jiwer

In [2]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import get_linear_schedule_with_warmup
from datasets import load_dataset, load_metric
import torch
from sacrebleu import corpus_bleu
from sacrebleu import corpus_ter
import pandas as pd
from datasets import Dataset
import evaluate
import nltk
import unicodedata
import jiwer

In [3]:
def preprocess_text(text):
    # Lowercase the text
    text = text.lower()
    # Normalize unicode characters
    text = unicodedata.normalize("NFKD", text)
    return text

In [ ]:
# Tokenizer
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

# Load data from files
with open("train.en", encoding="utf-8") as train_en, \
     open("train.hi", encoding="utf-8") as train_hi:

    # Read the lines from each file
    lines_en = train_en.readlines()
    lines_hi = train_hi.readlines()

    # Combine the lines into pairs and preprocess them
    data = [(preprocess_text(hi.strip()), preprocess_text(en.strip())) for hi, en in zip(lines_hi, lines_en)]

# Create a pandas DataFrame
train_df = pd.DataFrame(data, columns=["input_text","target_text"])

# Load data from files
with open("test.en", encoding="utf-8") as test_en, \
     open("test.hi", encoding="utf-8") as test_hi:

    # Read the lines from each file
    lines_en = test_en.readlines()
    lines_hi = test_hi.readlines()

    # Combine the lines into pairs and preprocess them
    data = [(preprocess_text(hi.strip()), preprocess_text(en.strip())) for hi, en in zip(lines_hi, lines_en)]

# Create a pandas DataFrame
test_df = pd.DataFrame(data, columns=["input_text","target_text"])

# Load data from files
with open("valid.en", encoding="utf-8") as valid_en, \
     open("valid.hi", encoding="utf-8") as valid_hi:

    # Read the lines from each file
    lines_en = valid_en.readlines()
    lines_hi = valid_hi.readlines()

    # Combine the lines into pairs and preprocess them
    data = [(preprocess_text(hi.strip()), preprocess_text(en.strip())) for hi, en in zip(lines_hi, lines_en)]

# Create a pandas DataFrame
valid_df = pd.DataFrame(data, columns=["input_text","target_text"])

# print(train_df)
# print(test_df)
# print(valid_df)

# Convert DataFrames to Datasets
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
valid_dataset = Dataset.from_pandas(valid_df)

In [4]:
#Tokenize function
def tokenize_data(example):
    input = tokenizer(example["input_text"], max_length=128, truncation=True, padding="max_length")
    target = tokenizer(example["target_text"], max_length=128, truncation=True, padding="max_length")
    return {"input_ids": input["input_ids"],
            "attention_mask": input["attention_mask"],
            "labels": target["input_ids"]}


In [ ]:
# Map the tokenization function to the datasets
tokenized_train_dataset = train_dataset.map(tokenize_data, batched=True, remove_columns=["input_text", "target_text"])
tokenized_valid_dataset = valid_dataset.map(tokenize_data, batched=True, remove_columns=["input_text", "target_text"])

Map:   0%|          | 0/21293 [00:00<?, ? examples/s]

Map:   0%|          | 0/942 [00:00<?, ? examples/s]

In [ ]:
# Model
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    evaluation_strategy="epoch",
    logging_dir="./logs",
    save_strategy="epoch",
    num_train_epochs=3,
    learning_rate=1e-5, # lower learning rate
    weight_decay=0.03,
    warmup_steps=2000, # increase warmup steps
    save_total_limit=3,
    lr_scheduler_type="linear", # use linear learning rate schedule
    load_best_model_at_end=True,
    metric_for_best_model="loss", # use validation loss for early stopping
)


# Initialize the trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_valid_dataset,
    tokenizer=tokenizer,
)

# Start training
trainer.train()

Make sure to replace the dataset file paths with the paths to your dataset files. This code snippet will fine-tune the mBART model for 3 epochs with a learning rate of 5e-5 and a batch size of 4. You can adjust these hyperparameters based on your computational resources and the size of your dataset.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!mkdir -p mbart
%cd "mbart"

/content/mbart


In [ ]:
!mkdir -p results

In [ ]:
# !cp -r '/content/mbart/results/checkpoint-7986' '/content/drive/MyDrive/latest_checkpoints'
!cp -r '/content/drive/MyDrive/latest_checkpoints' '/content/mbart/results/checkpoint-7986'

In [ ]:
pwd

In [5]:
# Load the fine-tuned model and tokenizer
model = MBartForConditionalGeneration.from_pretrained("./CMU_PHINC_HINGE_NEW_Cleaned_LC_28.57")  # Replace XXXX with the desired checkpoint number
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

In [6]:
# Load data from files
test_data = pd.read_csv("28.57/test.hi", sep="\t", header=None, names=["input_text"])
test_labels = pd.read_csv("28.57/test.en", sep="\t", header=None, names=["target_text"])

# Combine input and target text into single DataFrames
test_df = pd.concat([test_data, test_labels], axis=1)

test_df = test_df.dropna()

# Convert DataFrames to Datasets
test_dataset = Dataset.from_pandas(test_df)

# Map the tokenization function to the test dataset
tokenized_test_dataset = test_dataset.map(tokenize_data, batched=True, remove_columns=["input_text", "target_text"])

# Extract Hinglish and English sentences
test_hinglish = test_data["input_text"].apply(preprocess_text).tolist()
test_english = test_labels["target_text"].apply(preprocess_text).tolist()


# Reduce the size of the test data (e.g., use only the first 100 sentences)
test_hinglish_subset = test_hinglish[:15]
test_english_subset = test_english[:15]

Map:   0%|          | 0/960 [00:00<?, ? examples/s]

In [7]:
translations = []
for text in test_hinglish:
    inputs = tokenizer(text, return_tensors="pt", max_length=128, truncation=True)
    outputs = model.generate(**inputs)
    translation = tokenizer.decode(outputs[0], skip_special_tokens=True)
    translations.append(translation)


for i in range(len(test_hinglish_subset)):
  print("Test : ",test_english[i])
  print("Translation : ",translations[i])

# Calculate sacreBLEU score
bleu_score = corpus_bleu(translations, [test_english]).score
print(f'sacreBLEU score: {bleu_score}')

C:\Users\Mahendranath\anaconda3\lib\site-packages\transformers\generation\utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 200 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Test :  hi
Translation :  hi
Test :  marvel's the avengers is a 2012 american superhero film based on the marvel comics superhero team of the same name, produced by marvel studios and distributed by walt disneystudios motion pictures
Translation :  marvel's the avengers is the 2012 american superhero philm, which is based on the same name marvel comics superhero series, which is directed by marvel standyoj and is directed by wilfred dizanestodious.
Test :  hello. how are you? i am not entirely sure about what question to ask, so i'll just ask: do you think the critics were fair in their critique of the movie?
Translation :  hello. what's up? i don't really believe what i'm going to ask, but i'm sure sirph itana will ask: do you think the aalochak movies were spectacular?
Test :  i agree with them that ruffalo was great in the movie. 
Translation :  i agree with him that rafaalo was great in the movie.
Test :  i agree them great in the avengers   movie. 
Translation :  i agree, the aven

In [8]:
nltk.download('wordnet')

[nltk_data] Error loading wordnet: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>


False

In [9]:
meteor = evaluate.load('meteor')

[nltk_data] Error loading wordnet: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>
[nltk_data] Error loading punkt: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Mahendranath\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [10]:
# Create an empty list to store results
results = []

# Iterate through each prediction-reference pair
for prediction, reference in zip(translations, test_english):
    result = meteor.compute(predictions=[prediction], references=[reference])
    results.append(result['meteor'])

In [11]:
# Calculate the average METEOR score for all pairs
average_meteor = sum(results) / len(results)

print("Average METEOR score:", average_meteor)

Average METEOR score: 0.5307186053245752


In [12]:
print("METEOR scores for each pair:", results)

METEOR scores for each pair: [0.5, 0.6193693693693694, 0.46758130081300814, 0.8220833333333335, 0.8152173913043477, 0.8675523349436391, 0.5, 0.7559322033898305, 0.5, 0.9844782983615982, 0.864795918367347, 0.7575757575757577, 0.8675523349436391, 0.9993141289437586, 0.25, 0.7650100242692836, 0.2142857142857143, 0.46296296296296297, 0.6847545219638244, 0.9375, 0.7471655328798186, 0.2142857142857143, 0.22727272727272727, 0.28846153846153844, 0.25, 0.5888888888888889, 0.0, 0.8819444444444444, 0.0, 0.5569306930693068, 0.3254877014418999, 0.5645569620253165, 0.3333333333333333, 0.8819444444444444, 0.4206349206349207, 0.5968468468468469, 0.17241379310344826, 0.2808302808302808, 0.1923076923076923, 0.45731707317073167, 0.3292035398230088, 0.09174311926605505, 0.4505050505050505, 0.41396979503775616, 0.625, 0.5318681318681319, 0.7200590762620838, 0.5090725806451614, 0.9866898148148149, 0.9927268779852366, 0.40180180180180186, 0.08333333333333333, 0.9158653846153847, 0.701058201058201, 0.13513513

In [13]:
# Calculate TER using SacreBLEU
ter_score = corpus_ter(translations, [test_english])

print(f'TER score: {ter_score.score}')

TER score: 61.539153915391545


In [14]:
# Calculate WER
wer_score=0
for i in range(len(test_english)):
    wer = jiwer.wer(translations[i], test_english[i])
    wer_score+=wer

print("WER:", wer_score/len(test_english))

WER: 0.6808762139345391


In [ ]:
test_hi=['hello, aap khaana khaaye?']
# test_en=['hello, how are you']

In [ ]:
translations = []
for text in test_hi:
    inputs = tokenizer(text, return_tensors="pt", max_length=128, truncation=True)
    outputs = model.generate(**inputs)
    translation = tokenizer.decode(outputs[0], skip_special_tokens=True)
    translations.append(translation)


for i in range(len(test_hi)):
  # print("Test : ",test_en[i])
  print("Translation : ",translations[i])

# Calculate sacreBLEU score
# bleu_score = corpus_bleu(translations, [test_en]).score
# print(f'sacreBLEU score: {bleu_score}')

Translation :  hello, how are you?


In [ ]:
# # Generate translations
translations = []
for text in test_hinglish:
    inputs = tokenizer(text, return_tensors="pt", max_length=128, truncation=True)
    outputs = model.generate(**inputs)
    translation = tokenizer.decode(outputs[0], skip_special_tokens=True)
    translations.append(translation)


for i in range(15):
  print("Test : ",test_english[i])
  print("Translation : ",translations[i])

# Calculate sacreBLEU score
bleu_score = corpus_bleu(translations, [test_english]).score
print(f'sacreBLEU score: {bleu_score}')

# device = torch.cuda.current_device() if torch.cuda.is_available() else -1

# translated_sentences = []

# for sentence in test_hinglish_subset:
#     input_ids = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True).input_ids.to(device)
#     translations = model.generate(input_ids, num_return_sequences=1, max_new_tokens=200, no_repeat_ngram_size=2)
#     translated_text = tokenizer.batch_decode(translations, skip_special_tokens=True)
#     translated_sentences.extend(translated_text)


/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (200) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Test :  hi
Translation :  hi
Test :  marvel's the avengers is a 2012 american superhero film based on the marvel comics superhero team of the same name, produced by marvel studios and distributed by walt disneystudios motion pictures
Translation :  marvel's the avengers is a 2012 american superhero film based on the comics superhero series of the same name, starring marvel steadiway as nirmit and volt dizaneestoodios as the lead actors along with pitchers.
Test :  hello. how are you? i am not entirely sure about what question to ask, so i'll just ask: do you think the critics were fair in their critique of the movie?
Translation :  hi. how are you? i'm not sure exactly what to look at, so i'll look it up as well: do you think the characters were interesting in the movie adaptation?
Test :  i agree with them that ruffalo was great in the movie. 
Translation :  i agree they were great in the drama.
Test :  i agree them great in the avengers   movie. 
Translation :  i agree, the avengers 

In [ ]:
# bleu_score = sacrebleu.corpus_bleu(translated_sentences, [test_english_subset])
# print(f"BLEU Score: {bleu_score.score}")

Regarding the batch size and epochs, it depends on your computational resources and the size of your dataset. Since your dataset is relatively small, you can try increasing the number of epochs to 10 or even 20, and observe the performance on the validation set. If the performance on the validation set starts to degrade, you might be overfitting, and it’s time to stop training. As for the batch size, you can try increasing it to 8, 16, or 32, depending on your GPU memory. A larger batch size may speed up training and result in more stable convergence, but you may need to adjust the learning rate accordingly.

Remember that increasing the batch size may require more GPU memory, and increasing the number of epochs may lead to longer training times. You can experiment with different hyperparameters to find the best combination for your specific dataset and available resources.

Given that you have a 15 GB GPU, you can try the following hyperparameters:

per_device_train_batch_size: 8 or 16 (depending on memory usage)
num_train_epochs: 10 to 20 (monitor the validation loss to avoid overfitting)
learning_rate: 3e-5
Remember that increasing the batch size may require you to decrease the learning rate slightly. The suggested learning rate of 3e-5 should work well with a batch size of 8 or 16, but you might need to fine-tune the learning rate based on your specific case.

You can update the Seq2SeqTrainingArguments in the fine-tuning code with these new hyperparameters: